# Traitements des images

In [4]:
import os
from PIL import Image

Image.MAX_IMAGE_PIXELS = 100000000

Les images tests (résolution 0.1) sont contenues dans `/data/download_test/` et sont au format `EEEE-NNNN.tif`.

Les images font 10'000 x 10'000 pixels, il faut donc augmenter la limite de PIL pour que les images puissent être lues.

In [5]:
input_dir = "../data/download_test/"
output_dir = "../data/img_reduites/"

In [6]:
os.listdir(input_dir)

['2593-1119.tif',
 '2593-1120.tif',
 '2593-1121.tif',
 '2594-1120.tif',
 '2594-1121.tif']

In [7]:
new_width, new_height = 2000, 2000

In [14]:
coords = filename.split(".")[0]
ecoord, ncoord = coords.split('-')

In [16]:
ncoord

'1119'

In [17]:
for filename in os.listdir(input_dir):

    img = Image.open(os.path.join(input_dir, filename))

    coords = filename.split(".")[0]
    ecoord, ncoord = coords.split('-')
    
    for e in range(5):
        for n in range(5):
            left = e * new_width
            upper = n * new_height
            right = (e + 1) * new_width
            lower = (n + 1) * new_height

            # Crop the region from the original image
            new_image = img.crop((left, upper, right, lower))

            new_filename = f"{ecoord}{e*2}-{ncoord}{n*2}.tif"
            new_image.save(os.path.join(output_dir, new_filename))